<a href="https://colab.research.google.com/github/alephpt/AI_Projects/blob/main/GPT_Model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [15]:
import torch
import torch.nn as nn
from torch.nn import functional as fn
import requests

In [3]:
batch_size = 16 # Number of Sequences to Process in Parallel
block_size = 32 # Max Content Length for Predictions
max_iterations = 5000
eval_interval = 100
learning_rate = 1e-3
device = 'cuda' if torch.cuda.is_available() else 'cpu'
eval_iterations = 200 # Number of iterations before saving state
n_embed = 64 # Maps the input sequence into high-dimensional space
n_head = 4 # Attention head for multi-head attention layers
n_layers = 4
dropout = 0.0

In [57]:
torch.manual_seed(1337)
bible_url = 'https://www.o-bible.com/download/kjv.txt'
bible = requests.get(bible_url)
bible_text = bible.text.split("\n")
bible_text.remove(bible_text[0])

In [58]:
print(bible_text[0])

Ge1:1 In the beginning God created the heaven and the earth.


In [59]:
vocab = sorted(list(set(bible.text)))
vocab_size = len(vocab)
print("Vocab Size:", vocab_size)

Vocab Size: 73


In [66]:
stoi = { ch: i for i, ch in enumerate(vocab)}
itos = { i: ch for i, ch in enumerate(vocab)}
encode = lambda s: [stoi[c] for c in s]
decode = lambda l: "".join([itos[i] for i in l])

nums = encode("Today is the day.")
words = decode(nums)

print("nums", nums)
print("words", words)

nums [41, 61, 50, 47, 71, 1, 55, 65, 1, 66, 54, 51, 1, 50, 47, 71, 8]
words Today is the day.


In [ ]:
data = torch.tensor(encode(bible.text), dtype=torch.long)
print(data.shape, data.dtype)
print(data[:100])

In [ ]:
n = int(0.9 * len(data)) # train on the first 90%
training_data = data[:n]
validation_data = data[n:]
block_size = 8
training_data[:block_size + 1]
x = training_data[:block_size]
y = training_data[1:block_size + 1]

for t in range(block_size):
  context = x[:t+1]
  target = y[t]
  print(f"when input is {context} the target: {target}")

In [74]:
def get_batch(split):
  data = training_data if split == "train" else validation_data
  ix = torch.randint(len(data) - block_size, (batch_size,))
  x = torch.stack([data[i:i + block_size] for i in ix])
  y = torch.stack([data[i + 1:i + block_size + 1] for i in ix])
  return x, y

In [ ]:
xb, yb = get_batch('train')
print("inputs:")
print(xb.shape)
print(xb)
print("targets:")
print(yb.shape)
print(yb)

In [ ]:
for b in range(batch_size):
  for t in range(block_size):
    context = xb[b, :t + 1]
    target = yb[b, t]
    print(f"when input is {context.tolist()} the target: {target}")

In [80]:
@torch.no_grad()
def estimate_loss():
  out = {}
  model.eval()

  for split in ['train', 'eval']:
    losses = torch.zeros(eval_iterations)
    for k in range(eval_iterations):
      X, Y = get_batch(split)
      logits, loss = model(X, Y)
      losses[k] = loss.item()
    out[split] = losses.mean()

  model.train()
  return out